<a href="https://colab.research.google.com/github/sanahdia/scrapy-clustering/blob/main/HW3_clustering_sana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
from io import StringIO
import json

In [ ]:
df = pd.read_json('dataa.json', orient='records')

In [ ]:
x = (df['title'].fillna('') + ' ' + df['description'].fillna(''))
x = x.str.replace(r'[\t\n\r]', '').str.replace('\s+', ' ')
print(x)

0      Having previously vowed to pardon Jan. 6 riot...
1                                                      
2      On March 19, 1995, after a 21-month hiatus, M...
3      Having previously vowed to pardon Jan. 6 riot...
4      Last year, a team from New Jersey called Fair...
5      Paid loyalty programs are all the rage in the...
6     Social media influencer is charged with joinin...
7     Tuesday’s primaries include a key Senate race ...
8     Pro-Trump Michigan attorney arrested after hea...
9     Trump’s lawyers say it is impossible for him t...
10    Timelapse reveals Notre Dame’s new spire, rebu...
11    Woman wails over husband’s body following Isra...
12    Attempt to repeal ban on female genital mutila...
13    Tuesday’s primaries include a key Senate race ...
14    High-profile elections in Ohio could give Repu...
15    Trump backs Kevin McCarthy protege in Californ...
16    Bettors counting on upsets as they put money o...
17    March Madness brackets are here. Here’s ho

<ipython-input-31-99b83178118b>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  x = x.str.replace(r'[\t\n\r]', '').str.replace('\s+', ' ')


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=5)
df['title'].fillna('', inplace=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['title'])
terms = tfidf_vectorizer.get_feature_names_out()
print("List of terms included in the TF-IDF matrix:")
print(terms)
print("Size of the TF-IDF matrix:", tfidf_matrix.shape)

List of terms included in the TF-IDF matrix:
['for' 'in' 'to' 'trump']
Size of the TF-IDF matrix: (21, 4)


In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
kmeans = KMeans(init='k-means++')
kmeans.fit(tfidf_matrix)
cluster_counts = pd.Series(kmeans.labels_).value_counts().sort_index()
print("Number of elements in each cluster:")
print(cluster_counts)
print("Content of the clusters:")
for i, cluster_center in enumerate(kmeans.cluster_centers_):
    print("Cluster", i)
    cluster_indices = kmeans.labels_ == i
    cluster_samples = df[cluster_indices].head()
    print(cluster_samples)
    print()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Number of elements in each cluster:
0    10
1     1
2     3
3     2
4     1
5     1
6     2
7     1
dtype: int64
Content of the clusters:
Cluster 0
  title                                        description
0        Having previously vowed to pardon Jan. 6 riote...
1                                                         
2        On March 19, 1995, after a 21-month hiatus, Mi...
3        Having previously vowed to pardon Jan. 6 riote...
4        Last year, a team from New Jersey called Fairl...

Cluster 1
                                                title description
14  High-profile elections in Ohio could give Repu...            

Cluster 2
                                                title description
7   Tuesday’s primaries include a key Senate race ...            
13  Tuesday’s primaries include a key Senate race ...            
15  Trump backs Kevin McCarthy protege in Californ...            

Cluster 3
                                                title description
17 

In [ ]:
kmeans = KMeans(n_clusters=5, init='k-means++')

In [ ]:
kmeans.fit(tfidf_matrix)
cluster_counts = pd.Series(kmeans.labels_).value_counts().sort_index()
print("Number of elements in each cluster:")
print(cluster_counts)
print("Content of the clusters:")
for i, cluster_center in enumerate(kmeans.cluster_centers_):
    print("Cluster", i)
    cluster_indices = kmeans.labels_ == i
    cluster_samples = df[cluster_indices].head()
    print(cluster_samples)
    print()

Number of elements in each cluster:
0     2
1    10
2     3
3     4
4     2
dtype: int64
Content of the clusters:
Cluster 0
                                                title description
9   Trump’s lawyers say it is impossible for him t...            
12  Attempt to repeal ban on female genital mutila...            

Cluster 1
  title                                        description
0        Having previously vowed to pardon Jan. 6 riote...
1                                                         
2        On March 19, 1995, after a 21-month hiatus, Mi...
3        Having previously vowed to pardon Jan. 6 riote...
4        Last year, a team from New Jersey called Fairl...

Cluster 2
                                                title description
11  Woman wails over husband’s body following Isra...            
14  High-profile elections in Ohio could give Repu...            
20  Former Mormon bishop highlighted in AP investi...            

Cluster 3
                           

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw')
nltk.download('averaged_perceptron_tagger')
french_stopwords = set(stopwords.words('french'))
french_stemmer = SnowballStemmer('french')
def process_text(text):
    words = word_tokenize(text)
    filtered_stemmed = [french_stemmer.stem(word) for word in words if word.lower() not in french_stopwords]
    return ' '.join(filtered_stemmed)
df = pd.read_json('dataa.json', orient='records')
df = df.dropna()
df['processed_text'] = df['description'].apply(process_text)
if df['processed_text'].empty:
    print("No meaningful text found after preprocessing.")
else:
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3), analyzer='word')
    X = vectorizer.fit_transform(df['processed_text'])
    if X.shape[0] >= 10:
        kmeans = KMeans(n_clusters=10).fit(X)
        df['cluster'] = kmeans.labels_
        print(df.head())
    else:
        print("Number of samples is less than the number of clusters. Please reduce the number of clusters.")

No meaningful text found after preprocessing.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
